# 환경설정

In [35]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

# 크롬창 열기

In [22]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word_list = ['노량진수산시장 횟집', '송파구 횟집', '강서구 횟집', '강남구 횟집', '노원구 횟집', '은평구 횟집']

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_3180\2096135440.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로


In [23]:
key_word = ['노량진수산시장 횟집']

In [24]:
# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [25]:
def find_search_input(key_word):
    # 검색창 찾기
    time_wait(2, 'div.input_box > input.input_search')
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [38]:
# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경


In [27]:
# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [28]:
def find_store_list():
    # 매장 리스트
    store_list = driver.find_elements(By.CSS_SELECTOR,'._22p-O')
    # 페이지 리스트
    next_btn = driver.find_elements(By.CSS_SELECTOR,'._2ky45 > a')
    return store_list, next_btn

In [29]:
# 시작시간
start = time.time()
print('[크롤링 시작...]')

[크롤링 시작...]


In [30]:
def find_review_btn():
    menu_list = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_maintab a._2RG_o._tab-menu')
    if len(menu_list) ==7:
        return menu_list[-1]
    else:
        return menu_list[-2]

In [44]:
def crawling_visitor_review():
    #-----리뷰 버튼 클릭-------
    find_review_btn().click()
    SCROLL_PAUSE_SEC = 0.2
    
    #스크롤 한번 내리고
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    while True:
        try :
            #더보기 버튼 클릭
            driver.find_element(By.CSS_SELECTOR,'._3iTUo').click()
            # 끝까지 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            time.sleep(SCROLL_PAUSE_SEC)

        except:
            break

    #가게 리스트 마다 방문자 리뷰 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR,"div.faZHB > a > span")
    review_list = []
    for i in review_elements:
        review_list.append(i.text)
    
    return review_list


In [54]:
def crawling_blog_review():
        #-----리뷰 버튼 클릭-------
    find_review_btn().click()
    try :
        driver.find_element(By.CSS_SELECTOR, 'div._1tclc > div > a:nth-child(2)').click()
    except:
        print('블로그 리뷰가 없습니다.')
        return
    SCROLL_PAUSE_SEC = 0.2

    #스크롤 한번 내리고
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    while True:
        try :
            #더보기 버튼 클릭
            driver.find_element(By.CSS_SELECTOR,'._3iTUo').click()
            # 끝까지 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            time.sleep(SCROLL_PAUSE_SEC)

        except:
            break
    #가게 리스트 마다 방문자 리뷰 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR,'div._3dj5v > div._3jvFk')
    review_list = []
    for i in review_elements:
        review_list.append(i.text)
    
    return review_list


# 검색 지역 횟집의 리뷰를 크롤링.

In [58]:
review_dic = {}
def review_crawling(key_word):
    #검색창을 찾아서 서치
    find_search_input(key_word)
    # searchIframe 변경
    switch_frame('searchIframe')
    driver.implicitly_wait(2)
    # 페이지 버튼의 수
    pages = driver.find_elements(By.CSS_SELECTOR,'a._2tk2s') # 이건 필요
    for btn in range(0,4): #페이지 버튼
        pages[btn].click() #페이지 버튼 클릭
        #각 매장 요소들의 하이퍼링크 리스트
        page_down(40)
        driver.implicitly_wait(1)
        store_elements = driver.find_elements(By.CSS_SELECTOR,'div._3ZU00._1rBq3 > a:nth-child(1)')
        for data in range(len(store_elements)): #매장 리스트 만큼
            store_elements[data].click() #li의 페이지를 클릭
            driver.implicitly_wait(2) # 2초 휴식
            try:
                #다른 iframe으로 변경
                switch_frame('entryIframe')
                time_wait(2, '._3XamX')
        
                # -----매장명 가져오기-----
                store_name = driver.find_element(By.CSS_SELECTOR,'._3XamX').text
                # -----리뷰 크롤링---------
                review_dic[store_name] = crawling_visitor_review()
                review_dic[store_name] = review_dic[store_name]+crawling_blog_review()
                
            except:
                pass
            switch_frame('searchIframe')
        
            

In [59]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '노량진수상시장 횟집'
review_crawling(key_word)

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_3180\1201320596.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로


블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
._3XamX 태그를 찾지 못하였습니다.


MaxRetryError: HTTPConnectionPool(host='localhost', port=51353): Max retries exceeded with url: /session/c8af3fa00a8913a2fd6ea0f35b045160/frame (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000174110E5A00>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [57]:
review_dic

{'형제상회': ['노량진 수산시장의 신선한 모듬회를 인터넷으로 간 편하게 주문할 수 있으니 바쁜시간이나 근처에 호캉 스 할때 아주 만족스럽게 이용할 수 있었어요. ...',
  '내용 더보기',
  '역시 믿고가는 형제수산! 미리손질을 다 해주셔서 먹기 매우편하고 두께도 기본적으로 두꺼워서 좋네요! 매운탕은 두말하면 잔소리!',
  '진짜 최애집💛💛💛💛💛\n사진은 2인 6만원짜리입니당!\n기본 횟감이 두꺼우니 기본으로 먹는거 추천이요!',
  '2인 사만원 세트입니다 처음 가봤는데 굉장히 만족 해요 두껍게 썰어달라고 요청했고 연어는 조금만 달 라고 해서 조금만 주셨어요❤️ 재방문의사 10...',
  '내용 더보기',
  '정말 맛있었습니다. 동네에서는 절대 나올 수 없는 구성입니다. 회에 사용된 생선들이 굉장히 큰 놈 들 로만 구성하신게 눈에보였구요, 6만원을 내고 ...',
  '내용 더보기',
  '부모님과 서울여행 중 노량진 회센터에서 형제상회 먹게되었는데 ! 완전 강추 합니다 !! 양도 많고 부모님 도 다 만족하셨어요 5인가족이 아주 배불리 먹...',
  '내용 더보기',
  '노량진에서 제일 맛있어요',
  '회는 여기가 최고 👍 \n왕복 2시간 걸려도 꼭 형제상회로 갑니다.\n4인 11만+1인 4만',
  '한달에 1-2번은 꼭 들립니다.어제는 지인커플을 소개 해 주려고 같이 가서 먹었습니다.근데 제 기분탓이었 는지.?? 우선 진주식당에서 기다리는데 20분...',
  '내용 더보기',
  '와사비를 안넣어주셨어요. 외사비없이 회는 어떻게 맛있게 먹어요? 실망입니다',
  '믿고 찾는 형제상회입니다',
  '왜 이렇게 유명한데는 다 이유가 있음. 회맛있고 서더리도 완전 좋아서 매운탕도 맛있게 먹었어요.',
  '리뷰가 좋고 유명하다고 해서 와봤어요 근데 너무 실 망이네요 회가 시원하지 않고 맛이 없어요 맛에 비 해 너무 비싼게 아닌가 싶네요 신선도나 양은 ...',
  '내용 더보기',
  '도톰하고 싱싱해서 맛있었어요!!\n거의 기업같아요~~

In [ ]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '노량진수상시장 횟집'
time_wait(10, 'div.input_box > input.input_search')
# 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
driver.implicitly_wait(2)

In [40]:
switch_frame('searchIframe')
page_down(40)
sleep(1)
store_elements = driver.find_elements(By.CSS_SELECTOR,'div._3ZU00._1rBq3 > a:nth-child(1)')

In [41]:
store_elements[1].click()

In [42]:
review_dic = {}

#다른 iframe으로 변경
switch_frame('entryIframe')
time_wait(2, '._3XamX')

# -----매장명 가져오기-----
store_name = driver.find_element(By.CSS_SELECTOR,'._3XamX').text

# -----리뷰 크롤링---------
review_dic[store_name] = crawling_visitor_review()

In [43]:
review_dic

{'전라상회': ['지인들과 모임을 위해서 평소 방문하던 단골매장에 방문했습니다. 매번 방문할때마다 제가 원하는 종류 로 주문할 수 있어서 좋습니다. 이번엔 말이 필...',
  '내용 더보기',
  '어버이날이라 택배로 주문했어요! 매번 서울에서 본 가까지 가져왔었는데 이렇게도 받아볼 수 있자니 편 리하네요. 부모님이 웬만한 횟집에서 만족 못 ...',
  '내용 더보기',
  '노량진수산시장! 원래 다니던집 쫌 질려서 첨으로 전 라상회 방문했어요! 사장님 넘 친절하시고 잘 챙겨주 셔서 감사합니다. 잘먹었어요** 참치 먹고 싶...',
  '내용 더보기',
  '지인이 겨울에 대방어를 못 드셨다고해서 급하게 들\n른 전라상회. 잿방어와 참치로 한접시 요청드렸고 서\n비스로 올려주신 전갱이까지 최고였습니다. ...',
  '내용 더보기',
  '와우 어버이날 생각 못하고 갔다가 못살뻔했어요 오전에 주문하고 오후에 수령했는데 역시 너무 맛있어요 ㅠ 못샀으면 큰일날뻔요 ㅠ 짱맛!',
  '내 친구는 왜 이곳을 좋아하는가...',
  '싱싱한 회를 맛볼수 있어서 좋아요~^^',
  '진짜진짜진짜 너무 엄청 맛있게 먹었어요 서비스로 주신 라면사리와 수제비도 진짜 감사합니다',
  '노량진에서 항상 애용하는 횟집입니다',
  '노량진에서 여기만 가요!',
  '진짜 맛있어요!! 두툼한 회 맘껏 먹을수 있어 좋습니다!!',
  '지금까지 먹은 광어회 중에서 제일 맛있게 먹었어요~! 꼭 다시 방문할 거예요',
  '몇개는 맛있고 몇개는 좀 흐물거리고 우니는 너무 짜개져서 좀 아쉬웠네요',
  '비싸지만 존맛',
  '맛있어요',
  '좋아요 추천합니다',
  '😍',
  '요기만 가요',
  '회가 싱싱합니다',
  '방어를 좋아해서 그런지..입에서 녹았어요👍🏼👍🏼짱맛! 찾아보고 간곳은 아닌데 사람많고 키오스크가 있길 래 믿고 주문했습니닷 ㅎㅎ주말 저녁 7시쯤 주...',
  '내용 더보기',
  '회 정말 맛있어요',
  '항상 여기애용하고 있어요',
  '캐나다산 성게

In [205]:
#다른 iframe으로 변경
switch_frame('entryIframe')
time_wait(2, '._3XamX')

# -----매장명 가져오기-----
store_name = driver.find_element(By.CSS_SELECTOR,'._3XamX').text
crawling_visitor_review()

InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x009578B3+2193587]
	Ordinal0 [0x008F0681+1771137]
	Ordinal0 [0x008041A8+803240]
	Ordinal0 [0x00806BB4+814004]
	Ordinal0 [0x00806A72+813682]
	Ordinal0 [0x00806D00+814336]
	Ordinal0 [0x008321B5+991669]
	Ordinal0 [0x0083273B+993083]
	Ordinal0 [0x0085F7FC+1177596]
	Ordinal0 [0x0084D7F4+1103860]
	Ordinal0 [0x0085DAE2+1170146]
	Ordinal0 [0x0084D5C6+1103302]
	Ordinal0 [0x008277E0+948192]
	Ordinal0 [0x008286E6+952038]
	GetHandleVerifier [0x00C00CB2+2738370]
	GetHandleVerifier [0x00BF21B8+2678216]
	GetHandleVerifier [0x009E17AA+512954]
	GetHandleVerifier [0x009E0856+509030]
	Ordinal0 [0x008F743B+1799227]
	Ordinal0 [0x008FBB68+1817448]
	Ordinal0 [0x008FBC55+1817685]
	Ordinal0 [0x00905230+1856048]
	BaseThreadInitThunk [0x76BEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F475F4+228]
	RtlGetAppContainerNamedObjectPath [0x76F4

In [190]:
review_dic[store_name]= review_dic[store_name]+crawling_visitor_review()

In [186]:
[1,2,2,3] + [3,4,5]

[1, 2, 2, 3, 3, 4, 5]

In [191]:
review_dic[store_name]

['지인들과 모임을 위해서 평소 방문하던 단골매장에 방문했습니다. 매번 방문할때마다 제가 원하는 종류 로 주문할 수 있어서 좋습니다. 이번엔 말이 필...',
 '내용 더보기',
 '어버이날이라 택배로 주문했어요! 매번 서울에서 본 가까지 가져왔었는데 이렇게도 받아볼 수 있자니 편 리하네요. 부모님이 웬만한 횟집에서 만족 못 ...',
 '내용 더보기',
 '노량진수산시장! 원래 다니던집 쫌 질려서 첨으로 전 라상회 방문했어요! 사장님 넘 친절하시고 잘 챙겨주 셔서 감사합니다. 잘먹었어요** 참치 먹고 싶...',
 '내용 더보기',
 '지인이 겨울에 대방어를 못 드셨다고해서 급하게 들\n른 전라상회. 잿방어와 참치로 한접시 요청드렸고 서\n비스로 올려주신 전갱이까지 최고였습니다. ...',
 '내용 더보기',
 '와우 어버이날 생각 못하고 갔다가 못살뻔했어요 오전에 주문하고 오후에 수령했는데 역시 너무 맛있어요 ㅠ 못샀으면 큰일날뻔요 ㅠ 짱맛!',
 '내 친구는 왜 이곳을 좋아하는가...',
 '싱싱한 회를 맛볼수 있어서 좋아요~^^',
 '진짜진짜진짜 너무 엄청 맛있게 먹었어요 서비스로 주신 라면사리와 수제비도 진짜 감사합니다',
 '노량진에서 항상 애용하는 횟집입니다',
 '노량진에서 여기만 가요!',
 '진짜 맛있어요!! 두툼한 회 맘껏 먹을수 있어 좋습니다!!',
 '지금까지 먹은 광어회 중에서 제일 맛있게 먹었어요~! 꼭 다시 방문할 거예요',
 '몇개는 맛있고 몇개는 좀 흐물거리고 우니는 너무 짜개져서 좀 아쉬웠네요',
 '비싸지만 존맛',
 '맛있어요',
 '좋아요 추천합니다',
 '😍',
 '요기만 가요',
 '회가 싱싱합니다',
 '방어를 좋아해서 그런지..입에서 녹았어요👍🏼👍🏼짱맛! 찾아보고 간곳은 아닌데 사람많고 키오스크가 있길 래 믿고 주문했습니닷 ㅎㅎ주말 저녁 7시쯤 주...',
 '내용 더보기',
 '회 정말 맛있어요',
 '항상 여기애용하고 있어요',
 '캐나다산 성게알 35000원',
 '굿',
 '노량진 가면 여기서만 삽니다',